In [ ]:
import os
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import seaborn as sns
import re
import string
import shutil
import sys
import tqdm.notebook as tq
from collections import defaultdict

import torch
import torch.nn as nn

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [ ]:
%cd /content/drive/My Drive/research/Data

/content/drive/My Drive/research/Data


In [ ]:
df = pd.read_csv('sinhala_df_new.csv')

In [ ]:
columns = ['video_title','text','Label','Domain','About']

df = df[columns]

In [ ]:
df.Label.value_counts()

,count
Label,
Negative,2761
negative,1082
Positive,592
Neutral,466
positive,271
neutral,136
postive,14
Positve,11
negative,10


In [ ]:
df['Domain'] = df['Domain'].replace(['P', 'p','Political','Politician','political','Government','rajapaksha',' P'], 'political')
df['Domain'] = df['Domain'].replace(['non','Non','Non ','non ','Non-Political',' non'],'non')

In [ ]:
df['Combined'] = df['video_title'] + '.' + df['text']


In [ ]:
import re

def remove_emoji(string):
    if isinstance(string, str): # Check if the input is a string
        emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               "]+", flags=re.UNICODE)
        return emoji_pattern.sub(r'', string)
    else:
        return string # Return the original value if it's not a string

df['Combined'] = df['Combined'].apply(remove_emoji)

In [ ]:
import re

def remove_urls(text):
    if isinstance(text, str): # Check if the input is a string
        return re.sub(r'''(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’]))''', " ", text)
    else:
        return text # Return the original value if it's not a string

# Apply the function to the text column
df['Combined'] = df['Combined'].apply(remove_urls)

In [ ]:
def clean_data(text):
    if isinstance(text, str): # Check if the input is a string
        text = re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '', text) # Remove URLs

        text = re.sub(r'<.*?>', '', text)     # Remove HTML tags

        text = re.sub(r'@\\w+', '', text)  # Remove mentions

        text = re.sub(r'[^\u0D80-\u0DFF\s]', ' ', text) # Remove non-Sinhala characters (punctuation, numbers, etc.)
        return text
    else:
        return "" # Return an empty string if it's not a string

df['Combined'] = df['Combined'].apply(clean_data)

In [ ]:
df = df.dropna(subset=['Label'])

In [ ]:
df.Label.value_counts()

,count
Label,
Negative,2761
negative,1082
Positive,592
Neutral,466
positive,271
neutral,136
postive,14
Positve,11
negative,10


# Label 1 (Political / Non Political)

In [ ]:
df.Domain.value_counts()

,count
Domain,
political,3628
non,880


In [ ]:
df.Domain.value_counts()

,count
Domain,
political,3628
non,880


# Label 3 (Political Party / Politician )

In [ ]:
df.About.value_counts()

,count
About,
Ranil,552
Prasanna,428
NPP,402
Maithree,234
Other,161
...,...
Rajapakshat,1
sports minister,1
Premadasa,1


In [ ]:
df['About'] = df['About'].replace(['Mahinda', 'mahinda','Rajapaksha','Gota'], 'Rajapaksha')
df['About'] = df['About'].replace(['Government', 'government'], '225')
df['About'] = df['About'].replace(['Anura','JVP','NPP'], 'NPP')
df['About'] = df['About'].replace(['everyone', 'Everone','Everyone','all','225'], 'Everyone')
df['About'] = df['About'].replace(['Diana', 'FSP','Fonseka','sanath nishantha','vimal weeravansha','keheliya rabukwalla','Shantha bandara','Ravi Karunanayake','Yahapalanaya','Marikkar','Hareen','Maithripala','SLPP'], 'Other')

In [ ]:
df.About.value_counts()

,count
About,
Ranil,398
Everyone,123
Rajapaksha,120
Other,45
NPP,31
Sajith,27
UNP,20


# Label 2 (Positive/Negative/Neutral)

In [ ]:
df.Label.value_counts()

,count
Label,
Negative,1138
Neutral,350
Positive,238
Positve,10
positive,3


In [ ]:
df['Label'] = df['Label'].replace(['Positive','positive','Positve','P'], 'Positive')

In [ ]:
df.Label.value_counts()

,count
Label,
Negative,1138
Neutral,350
Positive,251


In [ ]:
df.to_csv('df_cleaned.csv',index=False)

In [ ]:
trainDF = pd.read_csv('df_cleaned.csv')

In [ ]:
from sklearn import model_selection, preprocessing # Import necessary modules


In [ ]:
trainDF.head()

,video_title,text,Label,Domain,About
0,"නොලැබීමට පරල වූ සෞඛ්‍ය වෘත්තීය සමිති... ""ඒ කා...",කාලකන්නි වෙිසිගෙ පුතාල හෙනමයි තොපිට වදින්නෙ ත...,Negative,Non political,NaN
1,"නොලැබීමට පරල වූ සෞඛ්‍ය වෘත්තීය සමිති... ""ඒ කා...","ඉල්ලුමට සැපයුම අනුව පඩි හැදෙන්නෙ, නැතුව ත්‍රස්...",Negative,Non political,NaN
2,හින්දු බැතිමතුන් ජයටම තෛපොංගල් සමරයි... ජාතික ...,ඊලග තයිපෝන්ගල් උත්සවය උපරිමයෙන් සාර්ථක ...,Positive,Political,Sajith
3,හින්දු බැතිමතුන් ජයටම තෛපොංගල් සමරයි... ජාතික ...,තයිපෝන්ගල් උත්සවය සාර්ථක වේවා sjb නායක තුමා...,Positive,Political,Sajith
4,උතුම් බුදු දහමට එරෙහි බලවේග ගැන මහානායක හිමිවර...,මේ රටේ මහා සංඝරත්නයට සමගියෙන් ඉන්න බැරි උනාම ඔ...,Neutral,Non political,NaN


In [ ]:
!pip install sinlingua

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.5/118.5 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.8/226.8 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.4/26.4 MB 46.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 48.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.0/17.0 MB 59.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 66.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.5/30.5 MB 38.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 60.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.8/32.8 MB 37.3 MB/s eta 0:00:00
   ━━━━━━━━━━━

# Stopwords removing

In [ ]:
from sinlingua.preprocessor.stopword_remover import StopWordRemover
stopword_remover = StopWordRemover()


In [ ]:
trainDF['text_new'] = trainDF['text'].apply(lambda x: stopword_remover.remove_stop_words(x))

trainDF.head()

,video_title,text,Label,Domain,About,text_new
0,"නොලැබීමට පරල වූ සෞඛ්‍ය වෘත්තීය සමිති... ""ඒ කා...",කාලකන්නි වෙිසිගෙ පුතාල හෙනමයි තොපිට වදින්නෙ ත...,Negative,Non political,NaN,කාලකන්නි වෙිසිගෙ පුතාල හෙනමයි තොපිට වදින්නෙ ත...
1,"නොලැබීමට පරල වූ සෞඛ්‍ය වෘත්තීය සමිති... ""ඒ කා...","ඉල්ලුමට සැපයුම අනුව පඩි හැදෙන්නෙ, නැතුව ත්‍රස්...",Negative,Non political,NaN,"ඉල්ලුමට සැපයුම පඩි හැදෙන්නෙ, නැතුව ත්‍රස්ථවාදී..."
2,හින්දු බැතිමතුන් ජයටම තෛපොංගල් සමරයි... ජාතික ...,ඊලග තයිපෝන්ගල් උත්සවය උපරිමයෙන් සාර්ථක ...,Positive,Political,Sajith,ඊලග තයිපෝන්ගල් උත්සවය උපරිමයෙන් සාර්ථක කරන්න අ...
3,හින්දු බැතිමතුන් ජයටම තෛපොංගල් සමරයි... ජාතික ...,තයිපෝන්ගල් උත්සවය සාර්ථක වේවා sjb නායක තුමා...,Positive,Political,Sajith,තයිපෝන්ගල් උත්සවය සාර්ථක වේවා sjb නායක තුමා සජිත්
4,උතුම් බුදු දහමට එරෙහි බලවේග ගැන මහානායක හිමිවර...,මේ රටේ මහා සංඝරත්නයට සමගියෙන් ඉන්න බැරි උනාම ඔ...,Neutral,Non political,NaN,රටේ සංඝරත්නයට සමගියෙන් ඉන්න බැරි උනාම ඔහොම තමය...


In [ ]:
# split the dataset into training and validation datasets
train_x, valid_x, train_y, valid_y = model_selection.train_test_split(trainDF['text_new'], trainDF['Label'])

# label encode the target variable
encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
valid_y = encoder.fit_transform(valid_y)

# Feature Engineering

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer


In [ ]:
# create a count vectorizer object
count_vect = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}',lowercase=False)
count_vect.fit(trainDF['text_new'].apply(' '.join))


CountVectorizer(lowercase=False, token_pattern='\\w{1,}')

In [ ]:
# transform the training and validation data using count vectorizer object
xtrain_count =  count_vect.transform(train_x)
xvalid_count =  count_vect.transform(valid_x)

In [ ]:
# Printing the identified Unique words along with their indices
print("Vocabulary: ", count_vect.vocabulary_)

# Encode the Document
vector = count_vect.transform(trainDF['text_new'])

Vocabulary:  {'ක': 88, 'ල': 116, 'ව': 117, 'ස': 120, 'ප': 108, 'ත': 102, 'හ': 121, 'න': 106, 'ම': 112, 'ද': 104, 'ග': 90, 'ය': 114, 'බ': 110, 'ඉ': 78, 'ඩ': 99, 'ර': 115, 'ථ': 103, 'ඕ': 86, 'අ': 74, 'ඔ': 85, 'ශ': 118, 'ධ': 105, 'D': 13, 'A': 10, 'T': 28, 'B': 11, 'ඊ': 79, 'උ': 80, 'ජ': 95, 'ආ': 75, 'ණ': 100, 's': 52, 'j': 43, 'b': 35, 'ට': 97, 'ඝ': 91, 'ඒ': 84, 'ළ': 122, 'ෂ': 119, 'භ': 111, 'ච': 93, 'ඡ': 94, 'එ': 83, 'ඇ': 76, 'ඛ': 89, 'r': 51, '2': 2, '4': 4, '6': 6, 'ඨ': 98, 'ඹ': 113, 'N': 23, 'G': 16, 'O': 24, 'ඌ': 81, 'E': 14, 'm': 46, 'a': 34, 'i': 42, 'l': 45, 'p': 49, 'h': 41, 'o': 48, 't': 53, 'c': 36, 'y': 58, 'w': 56, 'L': 21, 'n': 47, 'k': 44, 'ඳ': 107, '3': 3, '5': 5, '0': 0, '7': 7, 'ඵ': 109, 'e': 38, 'ඬ': 101, 'd': 37, 'q': 50, 'u': 54, 'I': 18, 'M': 22, 'F': 15, 'v': 55, 'g': 40, '1': 1, 'ෆ': 123, '9': 9, 'K': 20, 'z': 59, 'W': 31, 'U': 29, 'ඟ': 92, 'H': 17, 'V': 30, 'ஒ': 61, 'வ': 73, 'ர': 70, 'ம': 68, 'ற': 71, 'ய': 69, 'ப': 67, 'ட': 63, 'ள': 72, 'க': 62, 'எ': 60, 'ன': 66,

# TF-IDF Vectors as features

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer # Import the necessary class


In [ ]:
# word level tf-idf
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=5000)
tfidf_vect.fit(trainDF['text_new'])
xtrain_tfidf =  tfidf_vect.transform(train_x)
xvalid_tfidf =  tfidf_vect.transform(valid_x)

# ngram level tf-idf
tfidf_vect_ngram = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=5000)
tfidf_vect_ngram.fit(trainDF['text_new'])
xtrain_tfidf_ngram =  tfidf_vect_ngram.transform(train_x)
xvalid_tfidf_ngram =  tfidf_vect_ngram.transform(valid_x)

# characters level tf-idf
tfidf_vect_ngram_chars = TfidfVectorizer(analyzer='char', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=5000)
tfidf_vect_ngram_chars.fit(trainDF['text_new'])
xtrain_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(train_x)
xvalid_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(valid_x)

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:554: UserWarning: The parameter 'token_pattern' will not be used since 'analyzer' != 'word'
  warnings.warn(


In [ ]:
print("word level tf-idf_Vocabulary: ", tfidf_vect.vocabulary_)
print("ngram level tf-idf_Vocabulary: ", tfidf_vect_ngram.vocabulary_)
print("characters level tf-idf_Vocabulary: ", tfidf_vect_ngram_chars.vocabulary_)


word level tf-idf_Vocabulary:  {'ක': 1188, 'ලක': 1988, 'ව': 2031, 'ස': 2144, 'ප': 1625, 'ත': 1456, 'ල': 1987, 'හ': 2213, 'නම': 1594, 'වද': 2055, 'ග': 1283, 'තම': 1477, 'න': 1562, 'ය': 1841, 'බ': 1710, 'ඉල': 992, 'පය': 1657, 'පඩ': 1636, 'ද': 1502, 'රස': 1980, 'ථව': 1501, 'ඕ': 1176, 'යම': 1877, 'අගය': 778, 'අඩ': 785, 'යන': 1862, 'ඔපර': 1157, 'ශ': 2115, 'කරන': 1243, 'ධ': 1547, 'හරහ': 2247, 'ම': 1768, 'හදනව': 2228, 'ඉත': 977, 'dat': 219, 'bat': 164, 'ගග': 1285, 'ඉ': 968, 'ඊලග': 1012, 'තය': 1479, 'ගල': 1313, 'උත': 1032, 'සවය': 2200, 'උපර': 1042, 'මය': 1801, 'ර': 1911, 'ථ': 1498, 'කර': 1232, 'අන': 811, 'යය': 1884, 'සජ': 2149, 'මද': 1791, 'ආන': 908, 'ඩ': 1425, 'සම': 2168, 'ණය': 1452, 'ජනත': 1367, 'ලනය': 2011, 'sjb': 603, 'රට': 1932, 'ඝරත': 1332, 'නය': 1596, 'සමග': 2169, 'උන': 1037, 'ඔහ': 1174, 'ඒන': 1134, 'පළම': 1700, 'පක': 1626, 'ෂ': 2134, 'කරල': 1258, 'භ': 1762, 'රය': 1968, 'ජ': 1355, 'ච': 1336, 'ඡ': 1353, 'ශප': 2125, 'පස': 1686, 'යවත': 1892, 'වහන': 2111, 'අවව': 872, 'රක': 1912, 'ධර': 1560,

In [ ]:
print(xvalid_tfidf_ngram)

  (0, 4921)	0.12183838413364147
  (0, 4679)	0.11175329901637285
  (0, 4608)	0.1290719585356483
  (0, 4604)	0.14483806944990743
  (0, 4601)	0.18548005770244008
  (0, 4474)	0.15789992162048128
  (0, 4473)	0.14483806944990743
  (0, 4421)	0.1660538718006296
  (0, 4366)	0.14289834254343778
  (0, 4359)	0.08843963771817356
  (0, 4257)	0.16163555513416838
  (0, 4253)	0.1660538718006296
  (0, 4247)	0.18643042734015658
  (0, 4235)	0.1058355155161537
  (0, 4224)	0.07306210246153344
  (0, 4212)	0.12705131786687587
  (0, 4198)	0.08132073379493593
  (0, 4180)	0.17146145436586147
  (0, 4178)	0.09768156533600535
  (0, 3804)	0.14925638611636868
  (0, 3734)	0.16163555513416838
  (0, 3171)	0.15466396868160054
  (0, 2767)	0.1660538718006296
  (0, 2766)	0.15180965590247528
  (0, 2717)	0.10108293631850948
  :	:
  (434, 4601)	0.13596520139412505
  (434, 3937)	0.24344987166078294
  (434, 3934)	0.1453902962663304
  (434, 3916)	0.22675125197496443
  (434, 3909)	0.14630133858598074
  (434, 3897)	0.22256657339913

# Model Building

In [ ]:
def train_model(classifier, feature_vector_train, label, feature_vector_valid, is_neural_net=False):
    # fit the training dataset on the classifier
    classifier.fit(feature_vector_train, label)

    # predict the labels on validation dataset
    predictions = classifier.predict(feature_vector_valid)

    if is_neural_net:
        predictions = predictions.argmax(axis=-1)



    return metrics.accuracy_score(predictions, valid_y)


# Multinomial Naive Bayes

In [ ]:
from sklearn import naive_bayes
from sklearn import metrics

In [ ]:
# Naive Bayes on Count Vectors
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_count, train_y, xvalid_count)
print("NB, Count Vectors: ", accuracy)

# Naive Bayes on Word Level TF IDF Vectors
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf, train_y, xvalid_tfidf)
print ("NB, WordLevel TF-IDF: ", accuracy)

# Naive Bayes on Ngram Level TF IDF Vectors
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
print ("NB, N-Gram Vectors: ", accuracy)

# Naive Bayes on Character Level TF IDF Vectors
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf_ngram_chars, train_y, xvalid_tfidf_ngram_chars)
print ("NB, CharLevel Vectors: ", accuracy)

NB, Count Vectors:  0.6367816091954023
NB, WordLevel TF-IDF:  0.6597701149425287
NB, N-Gram Vectors:  0.6689655172413793
NB, CharLevel Vectors:  0.6827586206896552


In [ ]:
# Bernoulli Naive Bayes on Count Vectors
accuracy = train_model(naive_bayes.BernoulliNB(), xtrain_count, train_y, xvalid_count)
print("BernoulliNB, Count Vectors: ", accuracy)

# Bernoulli Naive Bayes on Word Level TF IDF Vectors
accuracy = train_model(naive_bayes.BernoulliNB(), xtrain_tfidf, train_y, xvalid_tfidf)
print ("BernoulliNB, WordLevel TF-IDF: ", accuracy)

# Bernoulli Naive Bayes on Ngram Level TF IDF Vectors
accuracy = train_model(naive_bayes.BernoulliNB(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
print ("BernoulliNB, N-Gram Vectors: ", accuracy)

# Bernoulli Naive Bayes on Character Level TF IDF Vectors
accuracy = train_model(naive_bayes.BernoulliNB(), xtrain_tfidf_ngram_chars, train_y, xvalid_tfidf_ngram_chars)
print ("BernoulliNB, CharLevel Vectors: ", accuracy)

BernoulliNB, Count Vectors:  0.6114942528735632
BernoulliNB, WordLevel TF-IDF:  0.6528735632183909
BernoulliNB, N-Gram Vectors:  0.6666666666666666
BernoulliNB, CharLevel Vectors:  0.6045977011494252


# Logistic Regression

In [ ]:
from sklearn import linear_model

In [ ]:
# Linear Classifier on Count Vectors
accuracy = train_model(linear_model.LogisticRegression(), xtrain_count, train_y, xvalid_count)
print ("LR, Count Vectors: ", accuracy)

# Linear Classifier on Word Level TF IDF Vectors
accuracy = train_model(linear_model.LogisticRegression(), xtrain_tfidf, train_y, xvalid_tfidf)
print ("LR, WordLevel TF-IDF: ", accuracy)

# Linear Classifier on Ngram Level TF IDF Vectors
accuracy = train_model(linear_model.LogisticRegression(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
print ("LR, N-Gram Vectors: ", accuracy)

# Linear Classifier on Character Level TF IDF Vectors
accuracy = train_model(linear_model.LogisticRegression(), xtrain_tfidf_ngram_chars, train_y, xvalid_tfidf_ngram_chars)
print ("LR, CharLevel Vectors: ", accuracy)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LR, Count Vectors:  0.6482758620689655
LR, WordLevel TF-IDF:  0.7057471264367816
LR, N-Gram Vectors:  0.6919540229885057
LR, CharLevel Vectors:  0.7126436781609196


# Support Vector Machine

In [ ]:
from sklearn import svm

In [ ]:
# SVM on Count Vectors
accuracy = train_model(svm.SVC(), xtrain_count, train_y, xvalid_count)
print ("SVM, Count Vectors: ", accuracy)

# SVM on Ngram Level TF IDF Vectors
accuracy = train_model(svm.SVC(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
print ("SVM, Ngram Level TF IDF Vectors: ", accuracy)

# SVM on Word Level TF IDF Vectors
accuracy = train_model(svm.SVC(), xtrain_tfidf, train_y, xvalid_tfidf)
print ("SVM, Word Level TF IDF Vectors: ", accuracy)

# SVM on Character Level TF IDF Vectors
accuracy = train_model(svm.SVC(), xtrain_tfidf_ngram_chars, train_y, xvalid_tfidf_ngram_chars)
print ("SVM, Character Level TF IDF Vectors: ", accuracy)

SVM, Count Vectors:  0.6252873563218391
SVM, Ngram Level TF IDF Vectors:  0.6804597701149425
SVM, Word Level TF IDF Vectors:  0.6873563218390805
SVM, Character Level TF IDF Vectors:  0.7011494252873564


# Random Forest - Bagging Model

In [ ]:
from sklearn import ensemble

In [ ]:

# RF on Count Vectors
accuracy = train_model(ensemble.RandomForestClassifier(), xtrain_count, train_y, xvalid_count)
print ("RF, Count Vectors: ", accuracy)

# RF on Word Level TF IDF Vectors
accuracy = train_model(ensemble.RandomForestClassifier(), xtrain_tfidf, train_y, xvalid_tfidf)
print ("RF, WordLevel TF-IDF: ", accuracy)

# RF on Ngram Level TF IDF Vectors
accuracy = train_model(ensemble.RandomForestClassifier(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
print ("RF, gram Level TF-IDF: ", accuracy)

# RF on Character Level TF IDF Vectors
accuracy = train_model(ensemble.RandomForestClassifier(), xtrain_tfidf_ngram_chars, train_y, xvalid_tfidf_ngram_chars)
print ("RF, Character Level TF IDF: ", accuracy)

RF, Count Vectors:  0.639080459770115
RF, WordLevel TF-IDF:  0.6896551724137931
RF, gram Level TF-IDF:  0.6896551724137931
RF, Character Level TF IDF:  0.7057471264367816


# Downsampling

In [ ]:
df_negative = trainDF[trainDF['Label'] == 'Negative'].sample(n = 251, random_state = 42)
df_neutral = trainDF[trainDF['Label'] == 'Neutral'].sample(n = 251, random_state = 42)
df_positive = trainDF[trainDF['Label'] == 'Positive'].sample(n = 251, random_state = 42)
df = pd.concat([df_negative,df_neutral,df_positive])

In [ ]:
# split the dataset into training and validation datasets
train_x, valid_x, train_y, valid_y = model_selection.train_test_split(df['text_new'], df['Label'])

# label encode the target variable
encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
valid_y = encoder.fit_transform(valid_y)

# Feature Engineering

In [ ]:
# create a count vectorizer object
count_vect = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}')
count_vect.fit(df['text_new'])

# transform the training and validation data using count vectorizer object
xtrain_count =  count_vect.transform(train_x)
xvalid_count =  count_vect.transform(valid_x)

In [ ]:
# Printing the identified Unique words along with their indices
print("Vocabulary: ", count_vect.vocabulary_)

# Encode the Document
vector = count_vect.transform(df['text'])

Vocabulary:  {'ප': 948, 'ඔය': 656, 'කර': 706, 'උ': 564, 'කවද': 733, 'ව': 1198, 'ග': 742, 'ය': 1081, 'ද': 873, 'ර': 1116, 'ල': 1166, 'බ': 997, 'හ': 1305, 'ඳ': 945, 'න': 911, 'බල': 1015, 'ම': 1036, 'ඕන': 667, 'ත': 843, 'ට': 803, 'රද': 1142, 'කලමන': 729, 'ක': 673, 'අවශ': 461, 'යත': 1090, 'වය': 1226, 'බලය': 1023, 'අප': 425, 'කට': 680, 'කක': 676, 'ෂගත': 1257, 'යව': 1105, 'ආග': 477, 'අත': 411, 'ජනත': 791, 'ජ': 785, 'බරදර': 1014, 'ඒ': 633, 'රය': 1153, 'ගන': 752, 'නව': 939, 'හරක': 1324, 'ඇමර': 520, 'ච': 773, 'දන': 879, 'වට': 1206, 'ඒක': 634, 'ඉව': 549, 'අල': 450, 'tip': 335, 'එ': 599, 'මද': 1053, 'මස': 1067, 'ස': 1263, 'තම': 854, 'අහ': 470, 'අස': 464, 'උන': 577, 'ඉවරය': 552, 'අන': 421, 'ගත': 750, 'ඉදප': 537, 'ජන': 789, 'දය': 886, 'එනක': 615, 'ඉ': 527, 'බටලන': 1000, 'තර': 857, 'දර': 888, 'අම': 431, 'මල': 1065, 'රබලව': 1149, 'අවක': 452, 'ශ': 1245, 'අධ': 419, 'පත': 958, 'ඹක': 1079, 'හර': 1323, '39': 31, 'රට': 1130, 'බඩ': 1002, 'ඩ': 822, 'කරල': 723, 'උද': 575, 'ධමනය': 906, 'ගනනය': 754, 'අමත': 432,

# TF IDF Vectors as Features

In [ ]:
# word level tf-idf
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=5000)
tfidf_vect.fit(df['text_new'])
xtrain_tfidf =  tfidf_vect.transform(train_x)
xvalid_tfidf =  tfidf_vect.transform(valid_x)

# ngram level tf-idf
tfidf_vect_ngram = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=5000)
tfidf_vect_ngram.fit(df['text_new'])
xtrain_tfidf_ngram =  tfidf_vect_ngram.transform(train_x)
xvalid_tfidf_ngram =  tfidf_vect_ngram.transform(valid_x)

# characters level tf-idf
tfidf_vect_ngram_chars = TfidfVectorizer(analyzer='char', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=5000)
tfidf_vect_ngram_chars.fit(df['text_new'])
xtrain_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(train_x)
xvalid_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(valid_x)

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:554: UserWarning: The parameter 'token_pattern' will not be used since 'analyzer' != 'word'
  warnings.warn(


In [ ]:
print("word level tf-idf_Vocabulary: ", tfidf_vect.vocabulary_)
print("ngram level tf-idf_Vocabulary: ", tfidf_vect_ngram.vocabulary_)
print("characters level tf-idf_Vocabulary: ", tfidf_vect_ngram_chars.vocabulary_)

word level tf-idf_Vocabulary:  {'ප': 948, 'ඔය': 656, 'කර': 706, 'උ': 564, 'කවද': 733, 'ව': 1198, 'ග': 742, 'ය': 1081, 'ද': 873, 'ර': 1116, 'ල': 1166, 'බ': 997, 'හ': 1305, 'ඳ': 945, 'න': 911, 'බල': 1015, 'ම': 1036, 'ඕන': 667, 'ත': 843, 'ට': 803, 'රද': 1142, 'කලමන': 729, 'ක': 673, 'අවශ': 461, 'යත': 1090, 'වය': 1226, 'බලය': 1023, 'අප': 425, 'කට': 680, 'කක': 676, 'ෂගත': 1257, 'යව': 1105, 'ආග': 477, 'අත': 411, 'ජනත': 791, 'ජ': 785, 'බරදර': 1014, 'ඒ': 633, 'රය': 1153, 'ගන': 752, 'නව': 939, 'හරක': 1324, 'ඇමර': 520, 'ච': 773, 'දන': 879, 'වට': 1206, 'ඒක': 634, 'ඉව': 549, 'අල': 450, 'tip': 335, 'එ': 599, 'මද': 1053, 'මස': 1067, 'ස': 1263, 'තම': 854, 'අහ': 470, 'අස': 464, 'උන': 577, 'ඉවරය': 552, 'අන': 421, 'ගත': 750, 'ඉදප': 537, 'ජන': 789, 'දය': 886, 'එනක': 615, 'ඉ': 527, 'බටලන': 1000, 'තර': 857, 'දර': 888, 'අම': 431, 'මල': 1065, 'රබලව': 1149, 'අවක': 452, 'ශ': 1245, 'අධ': 419, 'පත': 958, 'ඹක': 1079, 'හර': 1323, '39': 31, 'රට': 1130, 'බඩ': 1002, 'ඩ': 822, 'කරල': 723, 'උද': 575, 'ධමනය': 906, 'ගනනය'

In [ ]:
print(xvalid_tfidf_ngram)

  (0, 4446)	0.27732068427584344
  (0, 4438)	0.15923091512791973
  (0, 4415)	0.29010781938706337
  (0, 4412)	0.15339318186560785
  (0, 4368)	0.25244641029096915
  (0, 2941)	0.29010781938706337
  (0, 2925)	0.29010781938706337
  (0, 2918)	0.18630310771559946
  (0, 1506)	0.30147746202343695
  (0, 1074)	0.27732068427584344
  (0, 467)	0.27732068427584344
  (0, 465)	0.21570150464297433
  (0, 419)	0.29010781938706337
  (0, 417)	0.29010781938706337
  (0, 203)	0.25244641029096915
  (2, 3042)	0.38713429717501435
  (2, 2959)	0.38713429717501435
  (2, 2908)	0.2842468776061548
  (2, 2907)	0.22025755745687228
  (2, 1213)	0.21353549536763874
  (2, 1166)	0.20944320077696693
  (2, 1000)	0.3460204487960356
  (2, 996)	0.38713429717501435
  (2, 458)	0.32895665997909473
  (2, 457)	0.32197038923399773
  :	:
  (188, 2190)	0.1639837506880724
  (188, 2151)	0.15117232425027533
  (188, 2133)	0.09365182972197923
  (188, 1670)	0.08980628902593214
  (188, 1561)	0.15901400350412215
  (188, 1525)	0.1639837506880724
  

In [ ]:
def train_model(classifier, feature_vector_train, label, feature_vector_valid, is_neural_net=False):
    # fit the training dataset on the classifier
    classifier.fit(feature_vector_train, label)

    # predict the labels on validation dataset
    predictions = classifier.predict(feature_vector_valid)

    if is_neural_net:
        predictions = predictions.argmax(axis=-1)

    return metrics.accuracy_score(predictions, valid_y)

# Naive Bayes

In [ ]:
# Naive Bayes on Count Vectors
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_count, train_y, xvalid_count)
print("NB, Count Vectors: ", accuracy)

# Naive Bayes on Word Level TF IDF Vectors
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf, train_y, xvalid_tfidf)
print ("NB, WordLevel TF-IDF: ", accuracy)

# Naive Bayes on Ngram Level TF IDF Vectors
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
print ("NB, N-Gram Vectors: ", accuracy)

# Naive Bayes on Character Level TF IDF Vectors
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf_ngram_chars, train_y, xvalid_tfidf_ngram_chars)
print ("NB, CharLevel Vectors: ", accuracy)

NB, Count Vectors:  0.5555555555555556
NB, WordLevel TF-IDF:  0.582010582010582
NB, N-Gram Vectors:  0.5978835978835979
NB, CharLevel Vectors:  0.6190476190476191


In [ ]:
# Bernoulli Naive Bayes on Count Vectors
accuracy = train_model(naive_bayes.BernoulliNB(), xtrain_count, train_y, xvalid_count)
print("BernoulliNB, Count Vectors: ", accuracy)

# Bernoulli Naive Bayes on Word Level TF IDF Vectors
accuracy = train_model(naive_bayes.BernoulliNB(), xtrain_tfidf, train_y, xvalid_tfidf)
print ("BernoulliNB, WordLevel TF-IDF: ", accuracy)

# Bernoulli Naive Bayes on Ngram Level TF IDF Vectors
accuracy = train_model(naive_bayes.BernoulliNB(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
print ("BernoulliNB, N-Gram Vectors: ", accuracy)

# Bernoulli Naive Bayes on Character Level TF IDF Vectors
accuracy = train_model(naive_bayes.BernoulliNB(), xtrain_tfidf_ngram_chars, train_y, xvalid_tfidf_ngram_chars)
print ("BernoulliNB, CharLevel Vectors: ", accuracy)

BernoulliNB, Count Vectors:  0.5132275132275133
BernoulliNB, WordLevel TF-IDF:  0.5132275132275133
BernoulliNB, N-Gram Vectors:  0.5555555555555556
BernoulliNB, CharLevel Vectors:  0.582010582010582


# Logistic Regression

In [ ]:
# Linear Classifier on Count Vectors
accuracy = train_model(linear_model.LogisticRegression(), xtrain_count, train_y, xvalid_count)
print ("LR, Count Vectors: ", accuracy)

# Linear Classifier on Word Level TF IDF Vectors
accuracy = train_model(linear_model.LogisticRegression(), xtrain_tfidf, train_y, xvalid_tfidf)
print ("LR, WordLevel TF-IDF: ", accuracy)

# Linear Classifier on Ngram Level TF IDF Vectors
accuracy = train_model(linear_model.LogisticRegression(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
print ("LR, N-Gram Vectors: ", accuracy)

# Linear Classifier on Character Level TF IDF Vectors
accuracy = train_model(linear_model.LogisticRegression(), xtrain_tfidf_ngram_chars, train_y, xvalid_tfidf_ngram_chars)
print ("LR, CharLevel Vectors: ", accuracy)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LR, Count Vectors:  0.5238095238095238
LR, WordLevel TF-IDF:  0.5714285714285714
LR, N-Gram Vectors:  0.6190476190476191
LR, CharLevel Vectors:  0.6084656084656085


# Support Vector Machine

In [ ]:
# SVM on Count Vectors
accuracy = train_model(svm.SVC(), xtrain_count, train_y, xvalid_count)
print ("SVM, Count Vectors: ", accuracy)

# SVM on Ngram Level TF IDF Vectors
accuracy = train_model(svm.SVC(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
print ("SVM, Ngram Level TF IDF Vectors: ", accuracy)

# SVM on Word Level TF IDF Vectors
accuracy = train_model(svm.SVC(), xtrain_tfidf, train_y, xvalid_tfidf)
print ("SVM, Word Level TF IDF Vectors: ", accuracy)

# SVM on Character Level TF IDF Vectors
accuracy = train_model(svm.SVC(), xtrain_tfidf_ngram_chars, train_y, xvalid_tfidf_ngram_chars)
print ("SVM, Character Level TF IDF Vectors: ", accuracy)

SVM, Count Vectors:  0.5026455026455027
SVM, Ngram Level TF IDF Vectors:  0.5873015873015873
SVM, Word Level TF IDF Vectors:  0.5608465608465608
SVM, Character Level TF IDF Vectors:  0.6296296296296297


# Random Forest - Bagging Model

In [ ]:
# RF on Count Vectors
accuracy = train_model(ensemble.RandomForestClassifier(), xtrain_count, train_y, xvalid_count)
print ("RF, Count Vectors: ", accuracy)

# RF on Word Level TF IDF Vectors
accuracy = train_model(ensemble.RandomForestClassifier(), xtrain_tfidf, train_y, xvalid_tfidf)
print ("RF, WordLevel TF-IDF: ", accuracy)

# RF on Ngram Level TF IDF Vectors
accuracy = train_model(ensemble.RandomForestClassifier(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
print ("RF, gram Level TF-IDF: ", accuracy)

# RF on Character Level TF IDF Vectors
accuracy = train_model(ensemble.RandomForestClassifier(), xtrain_tfidf_ngram_chars, train_y, xvalid_tfidf_ngram_chars)
print ("RF, Character Level TF IDF: ", accuracy)

RF, Count Vectors:  0.5185185185185185
RF, WordLevel TF-IDF:  0.544973544973545
RF, gram Level TF-IDF:  0.5661375661375662
RF, Character Level TF IDF:  0.5555555555555556


# Translate into English

In [ ]:
pip install googletrans==4.0.0-rc1


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 3.2 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-4.0.0rc1-py3-none-any.whl size=17397 sha256=3276a2939122879ef8238cc44050da4cfc480cf4b46b4e85f04f8591fa69979a
  Stored in directory: /root/.cache/pip/wheels/c0/59/9f/7372f0cf70160fe61b528532e1a7c8498c4becd6bcffb022de
Successfully built googletrans
  Attempting uninstall: idna
    Found existing installation: idna 3.7
    Uninstalling idna-3.7:
      Success

In [ ]:
import pandas as pd
from googletrans import Translator

In [ ]:
 #Initialize the translator
translator = Translator()

# Function to translate text
def translate_text(text):
    try:
        translated = translator.translate(text, src='si', dest='en')
        return translated.text
    except Exception as e:
        print(f"Error translating text: {e}")
        return text

# Apply the function to the column
df['english_text'] = df['text'].apply(translate_text)


In [ ]:
df.to_csv('df_translated.csv',index=False)

In [ ]:
df_traslated = pd.read_csv('df_translated.csv')

In [ ]:
df_traslated.head()

,video_title,text,Label,Domain,About,english_text
0,"නොලැබීමට පරල වූ සෞඛ්‍ය වෘත්තීය සමිති... ""ඒ කා...",කාලකන්නි වෙිසිගෙ පුතාල හෙනමයි තොපිට වදින්නෙ ත...,Negative,Non political,NaN,"Wesari was a son-in-law, and you were bowed do..."
1,"නොලැබීමට පරල වූ සෞඛ්‍ය වෘත්තීය සමිති... ""ඒ කා...","ඉල්ලුමට සැපයුම අනුව පඩි හැදෙන්නෙ, නැතුව ත්‍රස්...",Negative,Non political,NaN,"If the offer is supplied, the payroll is done ..."
2,හින්දු බැතිමතුන් ජයටම තෛපොංගල් සමරයි... ජාතික ...,ඊලග තයිපෝන්ගල් උත්සවය උපරිමයෙන් සාර්ථක ...,Positive,Political,Sajith,Success the next Typhonegenial Ceremony to the...
3,හින්දු බැතිමතුන් ජයටම තෛපොංගල් සමරයි... ජාතික ...,තයිපෝන්ගල් උත්සවය සාර්ථක වේවා sjb නායක තුමා...,Positive,Political,Sajith,May the Typhonegegenial Festival be successful...
4,උතුම් බුදු දහමට එරෙහි බලවේග ගැන මහානායක හිමිවර...,මේ රටේ මහා සංඝරත්නයට සමගියෙන් ඉන්න බැරි උනාම ඔ...,Neutral,Non political,NaN,"It is when you can't stay united.Therefore, it..."


# English

In [ ]:
#Data cleaning and preprocessing
import re
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []

In [ ]:
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['message'][i])
    review = review.lower()
    review = review.split()

    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)


# Creating the Bag of Words model
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=2500)
X = cv.fit_transform(corpus).toarray()

y=pd.get_dummies(messages['label'])
y=y.iloc[:,1].values


# Train Test Split

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)

# Training model using Naive bayes classifier

from sklearn.naive_bayes import MultinomialNB
spam_detect_model = MultinomialNB().fit(X_train, y_train)

y_pred=spam_detect_model.predict(X_test)

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import SGDClassifier

In [ ]:
sgd = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, random_state=42, max_iter=5, tol=None)),
               ])
sgd.fit(train_x, train_y)

Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('clf',
                 SGDClassifier(alpha=0.001, max_iter=5, random_state=42,
                               tol=None))])

In [ ]:
sgd.predict(valid_x)

array([1, 1, 2, 1, 1, 2, 0, 0, 1, 0, 1, 0, 0, 0, 2, 1, 1, 2, 1, 2, 2, 2,
       2, 0, 2, 0, 1, 0, 0, 2, 1, 2, 0, 1, 2, 1, 2, 0, 0, 1, 2, 2, 0, 0,
       0, 0, 2, 0, 1, 1, 2, 2, 0, 2, 2, 2, 0, 0, 2, 2, 0, 1, 2, 1, 1, 0,
       2, 1, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 1, 2, 1, 1, 0, 0, 1, 1, 2, 0,
       2, 0, 2, 1, 1, 0, 0, 0, 2, 0, 2, 1, 0, 1, 2, 0, 2, 1, 1, 0, 1, 2,
       0, 0, 2, 0, 0, 0, 0, 1, 1, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 2, 2, 1,
       0, 0, 1, 0, 2, 0, 0, 0, 2, 2, 1, 2, 2, 1, 1, 2, 1, 0, 2, 2, 2, 2,
       2, 2, 0, 2, 1, 1, 1, 0, 1, 2, 0, 0, 2, 2, 0, 0, 1, 1, 1, 2, 0, 0,
       0, 2, 2, 1, 1, 0, 1, 0, 0, 0, 0, 2, 2])

In [ ]:
from pickle import load, dump
dump(sgd, open("model.pkl", "wb"))
model = load(open("model.pkl", "rb"))

NameError: name 'sgd' is not defined

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
from sklearn.metrics import classification_report
y_pred = model.predict(valid_x)
tags = ['Negative','Positive','Neutral']
print('Accuracy %s' % accuracy_score(y_pred, valid_y))

print(classification_report(valid_y, y_pred,target_names=tags))

Accuracy 0.5555555555555556
              precision    recall  f1-score   support

    Negative       0.57      0.67      0.61        63
    Positive       0.56      0.44      0.49        64
     Neutral       0.54      0.56      0.55        62

    accuracy                           0.56       189
   macro avg       0.56      0.56      0.55       189
weighted avg       0.56      0.56      0.55       189



In [ ]:
df = pd.read_csv('df_cleaned.csv')
df.head()

,video_title,text,Label,Domain,About
0,"නොලැබීමට පරල වූ සෞඛ්‍ය වෘත්තීය සමිති... ""ඒ කා...",කාලකන්නි වෙිසිගෙ පුතාල හෙනමයි තොපිට වදින්නෙ ත...,Negative,Non political,NaN
1,"නොලැබීමට පරල වූ සෞඛ්‍ය වෘත්තීය සමිති... ""ඒ කා...","ඉල්ලුමට සැපයුම අනුව පඩි හැදෙන්නෙ, නැතුව ත්‍රස්...",Negative,Non political,NaN
2,හින්දු බැතිමතුන් ජයටම තෛපොංගල් සමරයි... ජාතික ...,ඊලග තයිපෝන්ගල් උත්සවය උපරිමයෙන් සාර්ථක ...,Positive,Political,Sajith
3,හින්දු බැතිමතුන් ජයටම තෛපොංගල් සමරයි... ජාතික ...,තයිපෝන්ගල් උත්සවය සාර්ථක වේවා sjb නායක තුමා...,Positive,Political,Sajith
4,උතුම් බුදු දහමට එරෙහි බලවේග ගැන මහානායක හිමිවර...,මේ රටේ මහා සංඝරත්නයට සමගියෙන් ඉන්න බැරි උනාම ඔ...,Neutral,Non political,NaN


# Random Forest

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix


In [ ]:

trainDF.Label.value_counts()

,Label
Negative,1138
Neutral,350
Positive,251


In [ ]:
X = df['text']  # Features
y = df['Label']          # Labels

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


In [ ]:
vectorizer = TfidfVectorizer(max_features=5000)  # Adjust max_features as needed
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)


In [ ]:
!pip install imblearn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.3/258.3 kB 2.1 MB/s eta 0:00:00


In [ ]:
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train_tfidf, y_train)


RandomForestClassifier(random_state=42)

In [ ]:
y_pred = model.predict(X_test_tfidf)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))


Accuracy: 0.7068965517241379
Classification Report:
               precision    recall  f1-score   support

    Negative       0.77      0.90      0.83       347
     Neutral       0.49      0.29      0.36       100
    Positive       0.50      0.37      0.43        75

    accuracy                           0.71       522
   macro avg       0.59      0.52      0.54       522
weighted avg       0.68      0.71      0.68       522

Confusion Matrix:
 [[312  19  16]
 [ 59  29  12]
 [ 36  11  28]]


# SMOTE

In [ ]:
from imblearn.over_sampling import SMOTE


smote = SMOTE(random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train_tfidf, y_train)

# Check the new class distribution
print(pd.Series(y_train_res).value_counts())


Negative    791
Neutral     791
Positive    791
Name: Label, dtype: int64


In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
}

grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2)
grid_search.fit(X_train_tfidf, y_train)

print("Best Parameters:", grid_search.best_params_)


Fitting 5 folds for each of 108 candidates, totalling 540 fits
Best Parameters: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 10, 'n_estimators': 200}


In [ ]:
import joblib

joblib.dump(model, 'random_forest.pkl')
joblib.dump(vectorizer, 'tfidf_vectorizer.pkl')


['tfidf_vectorizer.pkl']

In [ ]:
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train_res, y_train_res)


RandomForestClassifier(random_state=42)

In [ ]:
y_pred = model.predict(X_test_tfidf)

# Print evaluation metrics
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))


Accuracy: 0.6494252873563219
Classification Report:
               precision    recall  f1-score   support

    Negative       0.78      0.78      0.78       347
     Neutral       0.37      0.34      0.35       100
    Positive       0.43      0.48      0.45        75

    accuracy                           0.65       522
   macro avg       0.53      0.53      0.53       522
weighted avg       0.65      0.65      0.65       522

Confusion Matrix:
 [[269  48  30]
 [ 48  34  18]
 [ 29  10  36]]


# Random Undersampling

In [ ]:
df_negative = df[df['Label'] == 'Negative'].sample(n = 251, random_state = 42)
df_neutral = df[df['Label'] == 'Neutral'].sample(n = 251, random_state = 42)
df_positive = df[df['Label'] == 'Positive'].sample(n = 251, random_state = 42)
df = pd.concat([df_negative,df_neutral,df_positive])

In [ ]:
df.Label.value_counts()

,Label
Negative,251
Neutral,251
Positive,251


In [ ]:
X = df['text']  # Features
y = df['Label']          # Labels

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
vectorizer = TfidfVectorizer(max_features=5000)  # Adjust max_features as needed
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

In [ ]:
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train_tfidf, y_train)

RandomForestClassifier(random_state=42)

In [ ]:
y_pred = model.predict(X_test_tfidf)

# Print evaluation metrics
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))


Accuracy: 0.5165562913907285
Classification Report:
               precision    recall  f1-score   support

    Negative       0.59      0.43      0.50        51
     Neutral       0.48      0.47      0.47        47
    Positive       0.50      0.64      0.56        53

    accuracy                           0.52       151
   macro avg       0.52      0.51      0.51       151
weighted avg       0.53      0.52      0.51       151

Confusion Matrix:
 [[22 11 18]
 [ 9 22 16]
 [ 6 13 34]]


# Multinomial Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)


In [ ]:
vectorizer = TfidfVectorizer(max_features=5000)  # Adjust max_features as needed
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)


In [ ]:
# By default, LogisticRegression uses the 'ovr' (One-vs-Rest) strategy for multi-class classification
# You can specify 'multinomial' to use Softmax Regression
model = LogisticRegression(max_iter=1000, random_state=42, multi_class='multinomial', solver='lbfgs')
model.fit(X_train_tfidf, y_train)


LogisticRegression(max_iter=1000, multi_class='multinomial', random_state=42)

In [ ]:
y_pred = model.predict(X_test_tfidf)

# Print evaluation metrics
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))


Accuracy: 0.5099337748344371
Classification Report:
               precision    recall  f1-score   support

    Negative       0.54      0.62      0.58        50
     Neutral       0.42      0.44      0.43        50
    Positive       0.59      0.47      0.52        51

    accuracy                           0.51       151
   macro avg       0.51      0.51      0.51       151
weighted avg       0.52      0.51      0.51       151

Confusion Matrix:
 [[31 15  4]
 [15 22 13]
 [11 16 24]]


In [ ]:
# Save the trained model
joblib.dump(model, 'logistic_regression_multiclass_model.pkl')

# Save the vectorizer
joblib.dump(vectorizer, 'tfidf_vectorizer.pkl')


['tfidf_vectorizer.pkl']

# SMOTE

In [ ]:
# Initialize SMOTE
smote = SMOTE(random_state=42)

# Apply SMOTE to the training data
X_train_res, y_train_res = smote.fit_resample(X_train_tfidf, y_train)

# Check the new class distribution
print(pd.Series(y_train_res).value_counts())


Positive    201
Negative    201
Neutral     201
Name: Label, dtype: int64


In [ ]:
# Train the Logistic Regression model on the balanced dataset
model = LogisticRegression(max_iter=1000, random_state=42, multi_class='multinomial', solver='lbfgs')
model.fit(X_train_res, y_train_res)


LogisticRegression(max_iter=1000, multi_class='multinomial', random_state=42)

In [ ]:
y_pred = model.predict(X_test_tfidf)

# Print evaluation metrics
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))


Accuracy: 0.5099337748344371
Classification Report:
               precision    recall  f1-score   support

    Negative       0.54      0.62      0.58        50
     Neutral       0.42      0.44      0.43        50
    Positive       0.59      0.47      0.52        51

    accuracy                           0.51       151
   macro avg       0.51      0.51      0.51       151
weighted avg       0.52      0.51      0.51       151

Confusion Matrix:
 [[31 15  4]
 [15 22 13]
 [11 16 24]]


# Downsampling

In [ ]:
# Initialize RandomUnderSampler
undersample = RandomUnderSampler(random_state=42)

# Apply downsampling to the training data
X_train_res, y_train_res = undersample.fit_resample(X_train_tfidf, y_train)

# Check the new class distribution
print(pd.Series(y_train_res).value_counts())


NameError: name 'RandomUnderSampler' is not defined

In [ ]:
# Train the Logistic Regression model on the balanced dataset
model = LogisticRegression(max_iter=1000, random_state=42, multi_class='multinomial', solver='lbfgs')
model.fit(X_train_res, y_train_res)


LogisticRegression(max_iter=1000, multi_class='multinomial', random_state=42)

In [ ]:
y_pred = model.predict(X_test_tfidf)

# Print evaluation metrics
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))


Accuracy: 0.5670498084291188
Classification Report:
               precision    recall  f1-score   support

    Negative       0.84      0.56      0.67       347
     Neutral       0.31      0.57      0.41       100
    Positive       0.41      0.61      0.49        75

    accuracy                           0.57       522
   macro avg       0.52      0.58      0.52       522
weighted avg       0.68      0.57      0.59       522

Confusion Matrix:
 [[193 109  45]
 [ 23  57  20]
 [ 14  15  46]]


# Naive Bayes

In [ ]:
from sklearn.naive_bayes import MultinomialNB

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)


In [ ]:
vectorizer = TfidfVectorizer(max_features=5000)  # Adjust max_features as needed
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)


In [ ]:
# Train the Multinomial Naive Bayes model on the balanced dataset
model = MultinomialNB()
model.fit(X_train_tfidf, y_train)


MultinomialNB()

In [ ]:
y_pred = model.predict(X_test_tfidf)

# Print evaluation metrics
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))


Accuracy: 0.7011494252873564
Classification Report:
               precision    recall  f1-score   support

    Negative       0.69      0.99      0.81       228
     Neutral       0.50      0.03      0.05        70
    Positive       1.00      0.32      0.48        50

    accuracy                           0.70       348
   macro avg       0.73      0.45      0.45       348
weighted avg       0.70      0.70      0.61       348

Confusion Matrix:
 [[226   2   0]
 [ 68   2   0]
 [ 34   0  16]]


# SMOTE

In [ ]:
# Initialize SMOTE
smote = SMOTE(random_state=42)

# Apply SMOTE to the training data
X_train_res, y_train_res = smote.fit_resample(X_train_tfidf, y_train)

# Check the new class distribution
print(pd.Series(y_train_res).value_counts())


Label
Negative    910
Positive    910
Neutral     910
Name: count, dtype: int64


In [ ]:
# Train the Multinomial Naive Bayes model on the balanced dataset
model = MultinomialNB()
model.fit(X_train_res, y_train_res)


MultinomialNB()

In [ ]:
y_pred = model.predict(X_test_tfidf)

# Print evaluation metrics
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))


Accuracy: 0.6954022988505747
Classification Report:
               precision    recall  f1-score   support

    Negative       0.78      0.82      0.80       228
     Neutral       0.49      0.36      0.41        70
    Positive       0.53      0.62      0.57        50

    accuracy                           0.70       348
   macro avg       0.60      0.60      0.59       348
weighted avg       0.69      0.70      0.69       348

Confusion Matrix:
 [[186  24  18]
 [ 36  25   9]
 [ 17   2  31]]


# Downsampling

https://github.com/Isurie/Text-Classification-Module/blob/master/Subjective_classification1.ipynb

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# For text preprocessing
import re
import nltk
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split

# For encoding labels
from sklearn.preprocessing import LabelEncoder

# For tokenization and padding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# For building the model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.utils import to_categorical

# For handling class imbalance
from sklearn.utils import class_weight

# Suppress TensorFlow warnings
import tensorflow as tf
tf.get_logger().setLevel('ERROR')

# Ensure NLTK data is downloaded
nltk.download('stopwords')

# ------------------------- Data Loading -------------------------
# Assuming your DataFrame is named 'df' with columns 'text' and 'label'
# Replace this with actual data loading code if necessary
# For example:
# df = pd.read_csv('your_dataset.csv')

# For demonstration, let's create a sample DataFrame
# data = {
#     'text': [
#         'මෙය නරක අදහසකි',
#         'මෙම නිෂ්පාදනය අගය කරන්නට ලැබුණි',
#         'මම මෙම සේවාව ගැන අනතුරු ඇගවීමක් පළ කරනවා',
#         'මෙය සාමාන්‍ය පරිච්ඡේදයකි',
#         'මෙම නිෂ්පාදනය අනර්ඝය'
#     ],
#     'label': ['negative', 'positive', 'negative', 'neutral', 'positive']
# }
# df = pd.DataFrame(data)

# ------------------------- Data Preprocessing -------------------------
# Define Sinhala stopwords (you might need to expand this list)
#sinhala_stopwords = ['ඇතුළත්', 'සඳහා', 'ඒ', 'අද']

def clean_text(text):
    # Remove non-alphabetic characters
    text = re.sub(r'[^අ-උැ-ෟ ]', '', text)
    # Convert to lowercase
    text = text.lower()
    # Remove stopwords
    tokens = text.split()
    tokens = [word for word in tokens if word not in sinhala_stopwords]
    return ' '.join(tokens)

# df['cleaned_text'] = df['text'].apply(clean_text)

# ------------------------- Label Encoding -------------------------
label_encoder = LabelEncoder()
df['encoded_label'] = label_encoder.fit_transform(df['Label'])
num_classes = len(label_encoder.classes_)

# ------------------------- Tokenization and Padding -------------------------
# Define parameters
max_words = 5000
max_len = 100
embedding_dim = 128

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(df['text'])
sequences = tokenizer.texts_to_sequences(df['text'])
padded_sequences = pad_sequences(sequences, maxlen=max_len, padding='post')

# ------------------------- Train-Test Split -------------------------
X = padded_sequences
y = to_categorical(df['encoded_label'], num_classes=num_classes)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# ------------------------- Handling Class Imbalance -------------------------
# Compute class weights
encoded_labels = np.argmax(y_train, axis=1)
class_weights = class_weight.compute_class_weight(class_weight='balanced',
                                                  classes=np.unique(encoded_labels),
                                                  y=encoded_labels)
class_weights = dict(enumerate(class_weights))

# ------------------------- Building the Model -------------------------
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=embedding_dim, input_length=max_len))
model.add(LSTM(64, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(num_classes, activation='softmax'))
learning_rate = 0.001
optimizer = Adam(learning_rate=learning_rate)

model.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

model.summary()

# ------------------------- Training the Model -------------------------
epochs = 10
batch_size = 16

history = model.fit(X_train, y_train,
                    epochs=epochs,
                    batch_size=batch_size,
                    validation_split=0.1,
                    class_weight=class_weights)

# ------------------------- Evaluating the Model -------------------------
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f'Test Accuracy: {accuracy*100:.2f}%')

# ------------------------- Plotting Training History -------------------------
plt.figure(figsize=(12,4))

# Plot accuracy
plt.subplot(1,2,1)
plt.plot(history.history['accuracy'], label='Train')
plt.plot(history.history['val_accuracy'], label='Validation')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

# Plot loss
plt.subplot(1,2,2)
plt.plot(history.history['loss'], label='Train')
plt.plot(history.history['val_loss'], label='Validation')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.show()


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


NameError: name 'df' is not defined